# Financial News Sentiment Analysis

In [ ]:
import os
import numpy as np
import pandas as pd

from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

## Data

Dataset source: https://huggingface.co/datasets/financial_phrasebank

* 0 (negative)
* 1 (neutral)
* 2 (positive)

In [ ]:
fi_data = load_dataset('financial_phrasebank', 'sentences_50agree')
fi_data

In [ ]:
df = pd.DataFrame(fi_data['train'])
df.head()

In [ ]:
print('Negative'.ljust(15) + 'Neutral'.ljust(15) + 'Positive'.ljust(15))
print('-'*45)
print(f"{len(df[df['label']==0])/len(df):.3g}".ljust(15) + f"{len(df[df['label']==1])/len(df):.3g}".ljust(15) + f"{len(df[df['label']==2])/len(df):.3g}".ljust(15))

### Split Dataset into Train and Test

In [ ]:
fi_data = fi_data['train'].train_test_split(test_size=0.2, shuffle=True, seed=123)
train_data = fi_data['train']
test_data = fi_data['train']
fi_data

### Preprocessing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)
tok_train = train_data.map(preprocess_function, batched=True)
tok_test = test_data.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [ ]:
tok_train

## Finetuning DistilBERT

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

In [ ]:
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [ ]:
training_args = TrainingArguments(
   output_dir='./',
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   use_cpu=True
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tok_train,
   eval_dataset=tok_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()